In [1]:
from ROOT import gSystem, gInterpreter, TChain, TH1F, TLorentzVector

# Path of Delphes directory 
# Personal laptop path
gSystem.AddDynamicPath("/home/nayan/MG5_aMC_v2_8_2/Delphes/")
# Cluster path
# ROOT.gSystem.AddDynamicPath("/home/johnson/MG5_aMC_v2_4_3/Delphes/")
gSystem.Load("libDelphes")

gInterpreter.Declare('#include "classes/DelphesClasses.h"')
gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')

from ROOT import ExRootTreeReader


Welcome to JupyROOT 6.22/05


In [2]:
filename = "tag_1_delphes_events.root"

# Create chain of root trees 
chain = TChain("Delphes")
chain.Add(filename)

# Create object of class ExRootTreeReader
myTree = ExRootTreeReader(chain)
Events = myTree.GetEntries()
Events

10000

In [3]:
# Get pointers to branches used in this analysis
# branchEvent = myTree.UseBranch("Event")
branchParticle = myTree.UseBranch("Particle")
# branchElectron = myTree.UseBranch("Electron")
# branchMuon = myTree.UseBranch("Muon")
# branchJet = myTree.UseBranch("Jet")
branchGenJets = myTree.UseBranch("GenJet")
# branchMissingET = myTree.UseBranch("MissingET")

In [8]:
Branch = branchParticle

status = []
event = []
s = 1
status_count = 0
# for n in range(Events):

myTree.ReadEntry(1)
    
for i in range(Branch.GetEntries()) :
    particle = Branch.At(i)
    if i == 0:

#         print('Event', n, ' Particle ', i, ' PID ', particle.PID, ' Status', particle.Status, 'E', particle.E, 'PT', particle.PT, ' Eta', particle.Eta, ' Phi', particle.Phi)

        particle1 = particle.P4()
    elif i == 12:
        particle2= particle.P4()


particle1.Eta()


100000000000.0

In [15]:
from ROOT import TMath

def comparison(A, B, Eta=True, Phi=True, Rapidity=True, R_Eta=True, R_Rap=True ):
    '''
    Given two particles will compute dEta, dPhi, dRapidty and two different versions of dR (one using dEta and one using dRapidity)
    A, B = Particle branch
    Eta, Phi, Rapidity, R = what comparisons to compute
    '''
    
    # Default values
    dEta, dPhi, dRapidity, dR_Eta, dR_Rap = 'NA', 'NA', 'NA', 'NA', 'NA'
    Pi = TMath.Pi()
    
    
    if Eta:
        dEta = A.Eta - B.Eta
        
    if Phi:
        dPhi = A.Phi - B.Phi
    
        # Constraining dPhi between -2Pi and 2Pi
        if dPhi < -Pi:
            dPhi = dPhi + 2*Pi
        elif dPhi > Pi:
            dPhi = dPhi - 2*Pi
    
    if Rapidity:
        dRapidity = A.Rapidity - B.Rapidity
    
    if R_Eta and Phi and Eta:
        dR_Eta = (dPhi**2 + dEta**2)**0.5
        
    if R_Rap and Phi and Rapidity:
        dR_Rap = (dPhi**2 + dRapidity**2)**0.5
    
    return (dEta, dPhi, dRapidity, dR_Eta, dR_Rap)